<a href="https://colab.research.google.com/github/Steve-YJ/Extracting-Important-Feature-of-Malimg-using-VAE/blob/master/01_MNIST_AAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reference
* god....!: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/aae/aae.py

### What I want to do
* run AAE 
* practicing train

* from 20.07.15.wed<br>
* prace & make my own work flow<br>
    * continue: https://github.com/Steve-YJ/Colab_Exercise/blob/master/Again_Training_Exp05_Just_20Epochs.ipynb

## 00. Mount Drive

* If you use colab, first mount driver

## 01. Library Import

## 02. Data Preprocessing
* Load dataset
* preprocess it
    * transforms
    * make custom dataset
    * train_test split
    * make train loader, test loader

* work Flow
    * transforms module 사용해서 image data compose
        * size 조정 및 normalize, tensor 변환
        * Image Folder를 이용해 dataload
        * dataset split: train dataset, test dataset
        * DataLoader로 batch단위 dataset 불러오기

# 03. Define Adversarial Variational AutoEncoder Class

* Encoder Class
* Decoder Class
* Discriminator Class

## 04. Training